In [1]:
import sys
import json
import pprint
import os

module_dir = os.path.join(os.path.abspath(''), '..')

if module_dir not in sys.path:
    sys.path.append(module_dir)

# Quickstart
Searches are performed through the `datasources.Manifest` object which acts as a manager for executing searches.  Datasource drivers are loaded into the library via a simple CLI.  Once loaded, the manifest contains a `dict` with each loaded datasource.

In [2]:
%%bash

cognition-datasources load -d Landsat8 -d Sentinel2

In [2]:
from datasources import Manifest

In [4]:
manifest = Manifest()

print(manifest)

{'Landsat8': <datasources.sources.Landsat8.Landsat8 object at 0x7fc2d07c3198>}


Each driver contains a `search` method with the pattern:

In [5]:
def search(spatial, temporal=None, properties=None, limit=10, *kwargs):
    pass

The input parameters are reflective of the STAC spec.  Spatial and temporal are standardized across all datasources while properties allows the user to query the datasource with any available STAC property (see the docs).  Please note that spatial is always required.

## Spatio-Temporal Query

In [8]:
# Query arguments
spatial = {"type": "Polygon",
            "coordinates": [
              [
                [
                  -101.28433227539062,
                  46.813218976041945
                ],
                [
                  -100.89431762695312,
                  46.813218976041945
                ],
                [
                  -100.89431762695312,
                  47.06450941441436
                ],
                [
                  -101.28433227539062,
                  47.06450941441436
                ],
                [
                  -101.28433227539062,
                  46.813218976041945
                ]
              ]
            ]
          }

temporal = ("2017-01-01", "2017-12-21")

# Perform the search
manifest['Landsat8'].search(spatial, temporal=temporal)

Creating a search appends the search parameters to the `manifest.searches` attribute.  The `manifest.execute` method will execute all searches within `manifest.searches` in parallel.  The response is a `dict` of GeoJSON Feature Collections where each feature is a STAC Item representing one returned asset

In [10]:
response = manifest.execute()

print("Number of returned assets: {}".format(len(response['Landsat8']['features'])))

Number of returned assets: 10


Looking at an individual STAC Item from the return:

In [14]:
stac_item = response['Landsat8']['features'][0]
print(json.dumps(stac_item, indent=2))

{
  "type": "Feature",
  "id": "LC80320282017353LGN00",
  "bbox": [
    -102.35362,
    44.90897,
    -99.21552,
    47.12249
  ],
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -101.6514535740085,
          47.12111867433461
        ],
        [
          -99.21640103633251,
          46.64713859617467
        ],
        [
          -99.92016479682236,
          44.91048259046667
        ],
        [
          -102.35198559503642,
          45.39215680874659
        ],
        [
          -101.6514535740085,
          47.12111867433461
        ]
      ]
    ]
  },
  "properties": {
    "collection": "landsat-8-l1",
    "datetime": "2017-12-19T17:29:43.300913+00:00",
    "eo:sun_azimuth": 162.91888311,
    "eo:sun_elevation": 18.72441598,
    "eo:cloud_cover": 87,
    "eo:row": "028",
    "eo:column": "032",
    "landsat:product_id": "LC08_L1TP_032028_20171219_20171224_01_T1",
    "landsat:scene_id": "LC80320282017353LGN00",
    "landsat:process

## Searching with STAC Properties
As mentioned earlier, STAC allows for querying against any STAC properties exposed by the dataset.  Let's look at the STAC properties of the above Landsat 8 asset:

In [15]:
print("STAC properties: {}".format(list(stac_item['properties'])))

STAC properties: ['collection', 'datetime', 'eo:sun_azimuth', 'eo:sun_elevation', 'eo:cloud_cover', 'eo:row', 'eo:column', 'landsat:product_id', 'landsat:scene_id', 'landsat:processing_level', 'landsat:tier', 'eo:epsg', 'eo:instrument', 'eo:off_nadir', 'eo:platform', 'eo:bands', 'eo:gsd']


Let's try finding all assets with less than 5% cloud cover using the **eo:cloud_cover** property.

In [18]:
# Create the search
manifest['Landsat8'].search(spatial, temporal=temporal, properties={'eo:cloud_cover': {'lt': 5}})

# Execute the search
response = manifest.execute()

# Verify the results
for feat in response['Landsat8']['features']:
    assert feat['properties']['eo:cloud_cover'] < 5

## Searching Multiple Sources
You can search multiple datasources at the same time in a similar fashion.  Please note that only the **spatial** and **temporal** parameters are standardized across all datasources.  The **properties** parameter varies between datasources as different datasources expose different metadata.

In [23]:
# Create a search for each source
for source in list(manifest):
    manifest[source].search(spatial, temporal=temporal)

print("Number of searches: {}".format(len(manifest.searches)))

Number of searches: 48


In [25]:
# Execute the search
response = manifest.execute()
for source in list(response):
    print("Found {} assets for {}".format(len(response[source]['features']), source))

Found 10 assets for Landsat8
Found 35 assets for NAIP
Found 10 assets for Sentinel1
Found 20 assets for Sentinel2
Found 1 assets for SRTM
Found 0 assets for CBERS
Found 4 assets for ElevationTiles


## Collections
Collections group available datasources into select groups for convenient querying across similar data sources.  Let's take a closer look at the EO (electro-optical) collection, which contains all datasources with R/G/B/NIR bands.  

In [6]:
from datasources.sources import collections

In [7]:
print("EO collection: {}".format([x.__name__ for x in collections.eo]))

EO collection: ['Landsat8']


In [10]:
# Load manifest with datasources according to tag:
manifest = Manifest(tags=['eo'])
pprint.pprint(manifest)

{'Landsat8': <datasources.sources.Landsat8.Landsat8 object at 0x7fc2d17997b8>}
